In [223]:
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


In [224]:
# have the warnings removed
import warnings
warnings.filterwarnings('ignore')

In [225]:
file = pd.read_csv("https://raw.githubusercontent.com/mengmeng2702/gaming_stats_analysis/working_T/for_ml_model.csv")

In [226]:

df = pd.DataFrame(data=file)
df.head()

,Unnamed: 0,appid,Game,release_date,english,developer,publisher,platforms,required_age,achievements,...,warhammer_40k,web_publishing,werewolves,western,word_game,world_war_i,world_war_ii,wrestling,zombies,e_sports
0,0,220,halflife 2,2004-11-16,1,Valve,Valve,windows;mac;linux,0,33,...,0,0,0,0,0,0,0,0,607,0
1,1,440,team fortress 2,2007-10-10,1,Valve,Valve,windows;mac;linux,0,520,...,0,0,0,0,0,0,0,0,0,0
2,2,500,left 4 dead,2008-11-17,1,Valve,Valve,windows;mac,0,73,...,0,0,0,0,0,0,0,0,1064,0
3,3,550,left 4 dead 2,2009-11-19,1,Valve,Valve,windows;mac;linux,0,70,...,0,0,0,0,0,0,0,0,5334,0
4,4,570,dota 2,2013-07-09,1,Valve,Valve,windows;mac;linux,0,0,...,0,0,0,0,0,0,0,0,0,4768


In [227]:
for col in df.columns:
    print(col)

Unnamed: 0
appid
Game
release_date
english
developer
publisher
platforms
required_age
achievements
positive_ratings
negative_ratings
average_playtime
median_playtime
owners
price
genre_concat
Unnamed: 0.1
Year_x
Publisher
Genre
Platform
Metascore
Avg_Userscore
No_Players
Rank
Month
Year_y
Hours_watched
Hours_Streamed
Peak_viewers
Peak_channels
Streamers
Avg_viewers
Avg_channels
Avg_viewer_ratio
1980s
1990s
2.5d
2d
2d_fighter
360_video
3d
3d_platformer
3d_vision
4_player_local
4x
6dof
atv
abstract
action
action_rpg
action_adventure
addictive
adventure
agriculture
aliens
alternate_history
america
animation_&_modeling
anime
arcade
arena_shooter
artificial_intelligence
assassin
asynchronous_multiplayer
atmospheric
audio_production
bmx
base_building
baseball
based_on_a_novel
basketball
batman
battle_royale
beat_em_up
beautiful
benchmark
bikes
blood
board_game
bowling
building
bullet_hell
bullet_time
crpg
capitalism
card_game
cartoon
cartoony
casual
cats
character_action_game
character_custo

In [228]:
#done once, file saved
# df.to_csv("Tony_for_ml_model.csv")

In [229]:
df

,Unnamed: 0,appid,Game,release_date,english,developer,publisher,platforms,required_age,achievements,...,warhammer_40k,web_publishing,werewolves,western,word_game,world_war_i,world_war_ii,wrestling,zombies,e_sports
0,0,220,halflife 2,2004-11-16,1,Valve,Valve,windows;mac;linux,0,33,...,0,0,0,0,0,0,0,0,607,0
1,1,440,team fortress 2,2007-10-10,1,Valve,Valve,windows;mac;linux,0,520,...,0,0,0,0,0,0,0,0,0,0
2,2,500,left 4 dead,2008-11-17,1,Valve,Valve,windows;mac,0,73,...,0,0,0,0,0,0,0,0,1064,0
3,3,550,left 4 dead 2,2009-11-19,1,Valve,Valve,windows;mac;linux,0,70,...,0,0,0,0,0,0,0,0,5334,0
4,4,570,dota 2,2013-07-09,1,Valve,Valve,windows;mac;linux,0,0,...,0,0,0,0,0,0,0,0,0,4768
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,235,711660,chuchel,2018-03-07,1,Amanita Design,Amanita Design,windows;mac,0,7,...,0,0,0,0,0,0,0,0,0,0
236,236,729040,borderlands,2019-04-03,1,Gearbox Software;Blind Squirrel Games,2K,windows,18,80,...,0,0,0,0,0,0,0,0,0,0
237,237,764790,the messenger,2018-08-30,1,Sabotage,Devolver Digital,windows,0,40,...,0,0,0,0,0,0,0,0,0,0
238,238,845070,the awesome adventures of captain spirit,2018-06-25,1,DONTNOD Entertainment,Square Enix,windows,0,0,...,0,0,0,0,0,0,0,0,0,0


In [230]:
df.drop("Unnamed: 0",inplace=True, axis=1)

In [231]:
column_names_list = df.columns.to_list()


In [232]:
li = []
def columns_ready_to_drop(col_name):
    #if column value type is int, continue the following, if not, ignore.
    if df[col_name].dtypes == "int64":

        #if dataframe col_name has 0 values more than 80% of the number of rows, save in a list, and ready to drop
        non_zero_rows = df[col_name].apply(lambda r: True if r > 0 else False)
        numberOfRows = len(non_zero_rows[non_zero_rows == True].index)
        #testing
        #print(numberOfRows)
        #When the percentage below is increasing, it means the column has to have less zero rows to be able to remain in the new database. 
        # 0.2 means if non-zero rows are only 20% or less of the total rows, the column will be add to the drop list.
        if numberOfRows <= 0.20 *len(df):
            li.append(col_name)
        else:
            pass
    else:
        pass
    return li

In [233]:
# use the function to run thru all columns
for col in column_names_list:
    columns_ready_to_drop(col)
    
print(li)


['1980s', '1990s', '2.5d', '2d', '2d_fighter', '360_video', '3d', '3d_platformer', '3d_vision', '4_player_local', '4x', '6dof', 'atv', 'abstract', 'action_rpg', 'action_adventure', 'addictive', 'agriculture', 'aliens', 'alternate_history', 'america', 'animation_&_modeling', 'anime', 'arcade', 'arena_shooter', 'artificial_intelligence', 'assassin', 'asynchronous_multiplayer', 'audio_production', 'bmx', 'base_building', 'baseball', 'based_on_a_novel', 'basketball', 'batman', 'battle_royale', 'beat_em_up', 'beautiful', 'benchmark', 'bikes', 'blood', 'board_game', 'bowling', 'building', 'bullet_hell', 'bullet_time', 'crpg', 'capitalism', 'card_game', 'cartoon', 'cartoony', 'casual', 'cats', 'character_action_game', 'character_customization', 'chess', 'choices_matter', 'choose_your_own_adventure', 'cinematic', 'city_builder', 'class_based', 'classic', 'clicker', 'co_op_campaign', 'cold_war', 'colorful', 'comedy', 'comic_book', 'competitive', 'conspiracy', 'controller', 'conversation', 'craf

In [234]:
#dropping columns when the dtypes of the columns is int64 and there are more than half of values are 0.
df_new = df.drop(columns=li)
df_new

,appid,Game,release_date,english,developer,publisher,platforms,required_age,achievements,positive_ratings,...,multiplayer,open_world,rpg,sandbox,sci_fi,shooter,singleplayer,story_rich,strategy,third_person
0,220,halflife 2,2004-11-16,1,Valve,Valve,windows;mac;linux,0,33,67902,...,488,0,0,0,1589,1049,1315,1116,0,0
1,440,team fortress 2,2007-10-10,1,Valve,Valve,windows;mac;linux,0,520,515879,...,10499,0,0,0,0,7857,0,0,0,0
2,500,left 4 dead,2008-11-17,1,Valve,Valve,windows;mac,0,73,17951,...,649,0,0,0,0,430,246,0,0,0
3,550,left 4 dead 2,2009-11-19,1,Valve,Valve,windows;mac;linux,0,70,251789,...,3176,0,0,0,0,1944,947,0,0,0
4,570,dota 2,2013-07-09,1,Valve,Valve,windows;mac;linux,0,0,863507,...,7747,0,2447,0,0,0,0,0,8242,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,711660,chuchel,2018-03-07,1,Amanita Design,Amanita Design,windows;mac,0,7,976,...,0,0,0,0,0,0,36,19,0,0
236,729040,borderlands,2019-04-03,1,Gearbox Software;Blind Squirrel Games,2K,windows,18,80,1648,...,0,0,37,0,0,0,0,0,0,0
237,764790,the messenger,2018-08-30,1,Sabotage,Devolver Digital,windows,0,40,1846,...,0,0,0,0,0,0,24,0,0,0
238,845070,the awesome adventures of captain spirit,2018-06-25,1,DONTNOD Entertainment,Square Enix,windows,0,0,8077,...,0,0,0,0,0,0,54,99,0,0


In [235]:
df_new.columns

Index(['appid', 'Game', 'release_date', 'english', 'developer', 'publisher',
       'platforms', 'required_age', 'achievements', 'positive_ratings',
       'negative_ratings', 'average_playtime', 'median_playtime', 'owners',
       'price', 'genre_concat', 'Unnamed: 0.1', 'Year_x', 'Publisher', 'Genre',
       'Platform', 'Metascore', 'Avg_Userscore', 'No_Players', 'Rank', 'Month',
       'Year_y', 'Hours_watched', 'Hours_Streamed', 'Peak_viewers',
       'Peak_channels', 'Streamers', 'Avg_viewers', 'Avg_channels',
       'Avg_viewer_ratio', 'action', 'adventure', 'atmospheric', 'co_op',
       'difficult', 'exploration', 'fps', 'fantasy', 'first_person',
       'great_soundtrack', 'indie', 'multiplayer', 'open_world', 'rpg',
       'sandbox', 'sci_fi', 'shooter', 'singleplayer', 'story_rich',
       'strategy', 'third_person'],
      dtype='object')

In [236]:
#put Game name as index
df_new = df_new.set_index("Game", drop = False, append=False, inplace=False)
df_new

,appid,Game,release_date,english,developer,publisher,platforms,required_age,achievements,positive_ratings,...,multiplayer,open_world,rpg,sandbox,sci_fi,shooter,singleplayer,story_rich,strategy,third_person
Game,,,,,,,,,,,,,,,,,,,,,
halflife 2,220,halflife 2,2004-11-16,1,Valve,Valve,windows;mac;linux,0,33,67902,...,488,0,0,0,1589,1049,1315,1116,0,0
team fortress 2,440,team fortress 2,2007-10-10,1,Valve,Valve,windows;mac;linux,0,520,515879,...,10499,0,0,0,0,7857,0,0,0,0
left 4 dead,500,left 4 dead,2008-11-17,1,Valve,Valve,windows;mac,0,73,17951,...,649,0,0,0,0,430,246,0,0,0
left 4 dead 2,550,left 4 dead 2,2009-11-19,1,Valve,Valve,windows;mac;linux,0,70,251789,...,3176,0,0,0,0,1944,947,0,0,0
dota 2,570,dota 2,2013-07-09,1,Valve,Valve,windows;mac;linux,0,0,863507,...,7747,0,2447,0,0,0,0,0,8242,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chuchel,711660,chuchel,2018-03-07,1,Amanita Design,Amanita Design,windows;mac,0,7,976,...,0,0,0,0,0,0,36,19,0,0
borderlands,729040,borderlands,2019-04-03,1,Gearbox Software;Blind Squirrel Games,2K,windows,18,80,1648,...,0,0,37,0,0,0,0,0,0,0
the messenger,764790,the messenger,2018-08-30,1,Sabotage,Devolver Digital,windows,0,40,1846,...,0,0,0,0,0,0,24,0,0,0


In [237]:
#create a new Dataframe that holds only the game names
game_names_df = df_new[["Game"]]
game_names_df

,Game
Game,
halflife 2,halflife 2
team fortress 2,team fortress 2
left 4 dead,left 4 dead
left 4 dead 2,left 4 dead 2
dota 2,dota 2
...,...
chuchel,chuchel
borderlands,borderlands
the messenger,the messenger


In [238]:
df_new.drop("Game",axis=1,inplace=True)
df_new

,appid,release_date,english,developer,publisher,platforms,required_age,achievements,positive_ratings,negative_ratings,...,multiplayer,open_world,rpg,sandbox,sci_fi,shooter,singleplayer,story_rich,strategy,third_person
Game,,,,,,,,,,,,,,,,,,,,,
halflife 2,220,2004-11-16,1,Valve,Valve,windows;mac;linux,0,33,67902,2419,...,488,0,0,0,1589,1049,1315,1116,0,0
team fortress 2,440,2007-10-10,1,Valve,Valve,windows;mac;linux,0,520,515879,34036,...,10499,0,0,0,0,7857,0,0,0,0
left 4 dead,500,2008-11-17,1,Valve,Valve,windows;mac,0,73,17951,948,...,649,0,0,0,0,430,246,0,0,0
left 4 dead 2,550,2009-11-19,1,Valve,Valve,windows;mac;linux,0,70,251789,8418,...,3176,0,0,0,0,1944,947,0,0,0
dota 2,570,2013-07-09,1,Valve,Valve,windows;mac;linux,0,0,863507,142079,...,7747,0,2447,0,0,0,0,0,8242,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chuchel,711660,2018-03-07,1,Amanita Design,Amanita Design,windows;mac,0,7,976,335,...,0,0,0,0,0,0,36,19,0,0
borderlands,729040,2019-04-03,1,Gearbox Software;Blind Squirrel Games,2K,windows,18,80,1648,920,...,0,0,37,0,0,0,0,0,0,0
the messenger,764790,2018-08-30,1,Sabotage,Devolver Digital,windows,0,40,1846,90,...,0,0,0,0,0,0,24,0,0,0


In [239]:
#check unique values for developers column
df_new.groupby("developer").count()

,appid,release_date,english,publisher,platforms,required_age,achievements,positive_ratings,negative_ratings,average_playtime,...,multiplayer,open_world,rpg,sandbox,sci_fi,shooter,singleplayer,story_rich,strategy,third_person
developer,,,,,,,,,,,,,,,,,,,,,
11 bit studios,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
2K Boston;2K Australia;Blind Squirrel;Feral Interactive (Mac),1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2K Czech;Feral Interactive (Mac),1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4A Games,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
AMPLITUDE Studios,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wargaming Group Limited,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
XLGAMES,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
id Software,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [240]:
#check unique values for platforms column
df_new.groupby("platforms").count()

,appid,release_date,english,developer,publisher,required_age,achievements,positive_ratings,negative_ratings,average_playtime,...,multiplayer,open_world,rpg,sandbox,sci_fi,shooter,singleplayer,story_rich,strategy,third_person
platforms,,,,,,,,,,,,,,,,,,,,,
windows,117,117,117,117,117,117,117,117,117,117,...,117,117,117,117,117,117,117,117,117,117
windows;linux,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
windows;mac,30,30,30,30,30,30,30,30,30,30,...,30,30,30,30,30,30,30,30,30,30
windows;mac;linux,91,91,91,91,91,91,91,91,91,91,...,91,91,91,91,91,91,91,91,91,91


In [241]:
df_new.dtypes

appid                 int64
release_date         object
english               int64
developer            object
publisher            object
platforms            object
required_age          int64
achievements          int64
positive_ratings      int64
negative_ratings      int64
average_playtime      int64
median_playtime       int64
owners               object
price               float64
genre_concat         object
Unnamed: 0.1          int64
Year_x                int64
Publisher            object
Genre                object
Platform             object
Metascore             int64
Avg_Userscore       float64
No_Players           object
Rank                  int64
Month                 int64
Year_y                int64
Hours_watched         int64
Hours_Streamed       object
Peak_viewers          int64
Peak_channels         int64
Streamers             int64
Avg_viewers           int64
Avg_channels          int64
Avg_viewer_ratio    float64
action                int64
adventure           

In [242]:
#check unique values for owners column
df_new.groupby("owners").count()
#owners to keep and get dummies

,appid,release_date,english,developer,publisher,platforms,required_age,achievements,positive_ratings,negative_ratings,...,multiplayer,open_world,rpg,sandbox,sci_fi,shooter,singleplayer,story_rich,strategy,third_person
owners,,,,,,,,,,,,,,,,,,,,,
100000-200000,13,13,13,13,13,13,13,13,13,13,...,13,13,13,13,13,13,13,13,13,13
1000000-2000000,59,59,59,59,59,59,59,59,59,59,...,59,59,59,59,59,59,59,59,59,59
10000000-20000000,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
100000000-200000000,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
200000-500000,49,49,49,49,49,49,49,49,49,49,...,49,49,49,49,49,49,49,49,49,49
2000000-5000000,51,51,51,51,51,51,51,51,51,51,...,51,51,51,51,51,51,51,51,51,51
20000000-50000000,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
50000-100000,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
500000-1000000,37,37,37,37,37,37,37,37,37,37,...,37,37,37,37,37,37,37,37,37,37


In [243]:
#check unique values for Hous_Streamed column
df_new.groupby("Hours_Streamed").count()
#owners to keep and get dummies

,appid,release_date,english,developer,publisher,platforms,required_age,achievements,positive_ratings,negative_ratings,...,multiplayer,open_world,rpg,sandbox,sci_fi,shooter,singleplayer,story_rich,strategy,third_person
Hours_Streamed,,,,,,,,,,,,,,,,,,,,,
10072 hours,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
101051 hours,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
10371 hours,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
10551 hours,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
10618 hours,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9694 hours,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
9751 hours,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
9840 hours,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [244]:
import re

In [245]:
#remove hours in Hours_Streamed column and create a new column called S_Hours
df_new["S_Hours"] = df_new["Hours_Streamed"].str.replace(r'\W*hours\W*','').astype(int)
df_new["S_Hours"]


Game
halflife 2                                    2357
team fortress 2                              25296
left 4 dead                                   1143
left 4 dead 2                                15441
dota 2                                      433397
                                             ...  
chuchel                                       1158
borderlands                                  80946
the messenger                                 9522
the awesome adventures of captain spirit     11897
mark of the ninja                              341
Name: S_Hours, Length: 240, dtype: int64

In [246]:
#create new column called year to get only the year from released_date column
#df_new["year"] = pd.DatetimeIndex(df["release_date"]).year
#df_new["year"]
#there is column Year_x already existing

In [247]:
#drop extra columns
df_new.drop(columns = ["genre_concat","publisher","developer","Publisher","Platform","Genre","No_Players","Hours_Streamed","release_date","appid"], axis=1, inplace=True)

In [248]:
df_new

,english,platforms,required_age,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,...,open_world,rpg,sandbox,sci_fi,shooter,singleplayer,story_rich,strategy,third_person,S_Hours
Game,,,,,,,,,,,,,,,,,,,,,
halflife 2,1,windows;mac;linux,0,33,67902,2419,691,402,10000000-20000000,7.19,...,0,0,0,1589,1049,1315,1116,0,0,2357
team fortress 2,1,windows;mac;linux,0,520,515879,34036,8495,623,20000000-50000000,0.00,...,0,0,0,0,7857,0,0,0,0,25296
left 4 dead,1,windows;mac,0,73,17951,948,897,278,5000000-10000000,7.19,...,0,0,0,0,430,246,0,0,0,1143
left 4 dead 2,1,windows;mac;linux,0,70,251789,8418,1615,566,10000000-20000000,7.19,...,0,0,0,0,1944,947,0,0,0,15441
dota 2,1,windows;mac;linux,0,0,863507,142079,23944,801,100000000-200000000,0.00,...,0,2447,0,0,0,0,0,8242,0,433397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chuchel,1,windows;mac,0,7,976,335,245,245,100000-200000,7.99,...,0,0,0,0,0,36,19,0,0,1158
borderlands,1,windows,18,80,1648,920,408,278,100000-200000,24.99,...,0,37,0,0,0,0,0,0,0,80946
the messenger,1,windows,0,40,1846,90,0,0,50000-100000,15.49,...,0,0,0,0,0,24,0,0,0,9522


In [249]:
df_new.dtypes

english               int64
platforms            object
required_age          int64
achievements          int64
positive_ratings      int64
negative_ratings      int64
average_playtime      int64
median_playtime       int64
owners               object
price               float64
Unnamed: 0.1          int64
Year_x                int64
Metascore             int64
Avg_Userscore       float64
Rank                  int64
Month                 int64
Year_y                int64
Hours_watched         int64
Peak_viewers          int64
Peak_channels         int64
Streamers             int64
Avg_viewers           int64
Avg_channels          int64
Avg_viewer_ratio    float64
action                int64
adventure             int64
atmospheric           int64
co_op                 int64
difficult             int64
exploration           int64
fps                   int64
fantasy               int64
first_person          int64
great_soundtrack      int64
indie                 int64
multiplayer         

In [250]:
#use get_dummies() to create variables for text features
X = pd.get_dummies(df_new,columns=["owners","platforms"])
X

,english,required_age,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,price,Unnamed: 0.1,Year_x,...,owners_2000000-5000000,owners_20000000-50000000,owners_50000-100000,owners_500000-1000000,owners_5000000-10000000,owners_50000000-100000000,platforms_windows,platforms_windows;linux,platforms_windows;mac,platforms_windows;mac;linux
Game,,,,,,,,,,,,,,,,,,,,,
halflife 2,1,0,33,67902,2419,691,402,7.19,20,2004,...,0,0,0,0,0,0,0,0,0,1
team fortress 2,1,0,520,515879,34036,8495,623,0.00,195,2007,...,0,1,0,0,0,0,0,0,0,1
left 4 dead,1,0,73,17951,948,897,278,7.19,532,2008,...,0,0,0,0,1,0,0,0,1,0
left 4 dead 2,1,0,70,251789,8418,1615,566,7.19,533,2009,...,0,0,0,0,0,0,0,0,0,1
dota 2,1,0,0,863507,142079,23944,801,0.00,454,2013,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chuchel,1,0,7,976,335,245,245,7.99,3043,2018,...,0,0,0,0,0,0,0,0,1,0
borderlands,1,18,80,1648,920,408,278,24.99,3039,2009,...,0,0,0,0,0,0,1,0,0,0
the messenger,1,0,40,1846,90,0,0,15.49,1940,2018,...,0,0,1,0,0,0,1,0,0,0


In [252]:
# Standardize the data with StandardScaler().
X_scaled = StandardScaler().fit_transform(X)
print(X_scaled[:])

[[ 0.         -0.55647772 -0.22476285 ... -0.09166985 -0.37796447
   1.27959472]
 [ 0.         -0.55647772  4.83597333 ... -0.09166985 -0.37796447
   1.27959472]
 [ 0.         -0.55647772  0.19090337 ... -0.09166985  2.64575131
  -0.78149744]
 ...
 [ 0.         -0.55647772 -0.15202126 ... -0.09166985 -0.37796447
  -0.78149744]
 [ 0.         -0.55647772 -0.56768747 ... -0.09166985 -0.37796447
  -0.78149744]
 [ 0.          1.61073773 -0.17280457 ... -0.09166985 -0.37796447
   1.27959472]]


In [253]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
df_pca= pca.fit_transform(X_scaled)
df_pca

array([[ 1.32200667e+00, -2.16076060e+00,  4.23615928e+00],
       [ 7.94408163e+00,  1.67474977e+00,  1.09569023e+01],
       [-3.84497129e-01, -1.47613570e-01,  2.43244509e+00],
       [ 3.00347970e+00, -4.51061817e-01,  4.20448447e+00],
       [ 1.73170372e+01,  1.26462200e+01, -2.95520063e+00],
       [ 2.54299794e+00, -2.58414313e+00,  3.82473132e+00],
       [ 3.51316766e+01,  1.96801875e+01,  2.95830181e+00],
       [-1.60930598e+00, -1.53067637e-01,  2.33009121e-01],
       [-1.77322075e+00,  3.76580298e-01,  1.46725496e+00],
       [-1.90874423e+00,  5.06874471e-01, -3.60208410e-01],
       [-1.54282507e+00, -1.75964710e-01,  2.71222465e+00],
       [-7.17872372e-01,  7.64469115e-01, -2.21481831e-01],
       [-1.56770145e+00, -6.32080483e-02,  8.89351087e-01],
       [-9.73612388e-01, -8.50124781e-01,  1.90657987e+00],
       [-1.02074054e+00, -6.51128177e-02,  2.04329266e+00],
       [-8.81032071e-01, -1.47603997e-01,  1.98569752e+00],
       [ 7.23324486e-01, -2.08420160e+00

In [254]:
# Create a DataFrame with the three principal components.
pcs_df= pd.DataFrame(
    data = df_pca, columns = ["PC 1","PC 2","PC 3"],index=X.index
)
pcs_df

,PC 1,PC 2,PC 3
Game,,,
halflife 2,1.322007,-2.160761,4.236159
team fortress 2,7.944082,1.674750,10.956902
left 4 dead,-0.384497,-0.147614,2.432445
left 4 dead 2,3.003480,-0.451062,4.204484
dota 2,17.317037,12.646220,-2.955201
...,...,...,...
chuchel,-1.845056,0.776215,-0.263958
borderlands,-0.293050,1.379910,-2.704964
the messenger,-1.581731,0.851804,-1.145192


In [255]:
# Create an elbow curve to find the best value for K.

inertia = []
k = list(range(1,11))
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

elbow_data = {"k":k,"inertia":inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k",y="inertia",xticks=k,title= "Elbow Curve")

:Curve   [k]   (inertia)

In [256]:
# Initialize the K-Means model.
model = KMeans(n_clusters=6,random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)
predictions

array([0, 4, 0, 0, 4, 0, 3, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0,
       2, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 1, 2, 0, 1, 2, 0, 0, 0, 0, 0,
       0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 2, 0, 2, 0,
       0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 2, 0, 2, 0, 0, 2, 0, 0, 0, 1, 2, 2,
       2, 5, 2, 2, 0, 0, 0, 0, 0, 0, 2, 0, 1, 2, 2, 2, 0, 2, 2, 2, 0, 2,
       1, 0, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 1, 2, 1, 2,
       2, 0, 2, 2, 2, 0, 0, 2, 2, 0, 1, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 2,
       2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 0, 2, 2, 0, 2,
       2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0],
      dtype=int32)

In [257]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([df_new,pcs_df],axis=1)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df["GameName"]=game_names_df

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(240, 51)


,english,platforms,required_age,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,...,singleplayer,story_rich,strategy,third_person,S_Hours,PC 1,PC 2,PC 3,GameName,Class
Game,,,,,,,,,,,,,,,,,,,,,
halflife 2,1,windows;mac;linux,0,33,67902,2419,691,402,10000000-20000000,7.19,...,1315,1116,0,0,2357,1.322007,-2.160761,4.236159,halflife 2,0
team fortress 2,1,windows;mac;linux,0,520,515879,34036,8495,623,20000000-50000000,0.00,...,0,0,0,0,25296,7.944082,1.674750,10.956902,team fortress 2,4
left 4 dead,1,windows;mac,0,73,17951,948,897,278,5000000-10000000,7.19,...,246,0,0,0,1143,-0.384497,-0.147614,2.432445,left 4 dead,0
left 4 dead 2,1,windows;mac;linux,0,70,251789,8418,1615,566,10000000-20000000,7.19,...,947,0,0,0,15441,3.003480,-0.451062,4.204484,left 4 dead 2,0
dota 2,1,windows;mac;linux,0,0,863507,142079,23944,801,100000000-200000000,0.00,...,0,0,8242,0,433397,17.317037,12.646220,-2.955201,dota 2,4
portal 2,1,windows;mac;linux,0,51,138220,1891,1102,520,10000000-20000000,7.19,...,1849,1084,650,0,7278,2.542998,-2.584143,3.824731,portal 2,0
counterstrike global offensive,1,windows;mac;linux,0,167,2644404,402313,22494,6502,50000000-100000000,0.00,...,0,0,5573,0,830105,35.131677,19.680187,2.958302,counterstrike global offensive,3
vampire the masquerade bloodlines,1,windows,0,0,6161,362,350,202,500000-1000000,14.99,...,355,217,0,102,625,-1.609306,-0.153068,0.233009,vampire the masquerade bloodlines,0
call of duty 2,1,windows;mac,0,0,2434,236,750,1041,200000-500000,14.99,...,107,10,0,0,898,-1.773221,0.376580,1.467255,call of duty 2,0


In [258]:
# Creating a 3D-Scatter with the PCA data and the clusters

fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="Class",
    symbol="Class",
    width=800,
    hover_name="GameName",
    hover_data=["GameName","S_Hours"]
    )
fig.update_layout(legend=dict(x=0,y=1))
fig.show()